In [53]:
import os

In [54]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [55]:
load_dotenv()

True

In [56]:
mykey=os.getenv("OPENAI_API_KEY")

In [57]:
llm=ChatOpenAI(openai_api_key=mykey,model_name="gpt-3.5-turbo",temperature=0.7)

In [66]:
with open("C:/Users/anshu/OneDrive/Desktop/chaku/GenAI/GenAI/Response.json","r") as f:
    json.load(f)

In [67]:
print(RESPONSE_JSON)

{'quiz': {'text': '{text}', 'subject': '{subject}', 'tone': '{tone}', 'questions': [{'question': 'Question 1 text based on the provided content', 'options': ['Option 1', 'Option 2', 'Option 3', 'Option 4'], 'correct_answer': 'Option 1'}, {'question': 'Question 2 text based on the provided content', 'options': ['Option 1', 'Option 2', 'Option 3', 'Option 4'], 'correct_answer': 'Option 2'}]}}


In [68]:
TEMPLATE="""
TEXT:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone}.\
Make sure the questions are not repeated and check all the questions to conforming.\
Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}
"""

In [69]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","RESPONSE_JSON"],
    template=TEMPLATE
)

In [70]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [71]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [72]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [73]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [74]:
generate_evalution_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","RESPONSE_JSON"],output_variables=["quiz","review"],verbose=True)

In [75]:
PATH="C:/Users/anshu/OneDrive/Desktop/chaku/GenAI/GenAI/data.txt"

In [76]:
with open(PATH,"r") as file:
    TEXT=file.read

In [77]:
print(TEXT)

<built-in method read of _io.TextIOWrapper object at 0x0000019A97288580>


In [78]:
TEXT
NUMBER=5
SUBJECT="AI"
TONE="Simple"
RESPONSE_JSON=RESPONSE_JSON

In [79]:
import json
json.dumps(RESPONSE_JSON)

'{"quiz": {"text": "{text}", "subject": "{subject}", "tone": "{tone}", "questions": [{"question": "Question 1 text based on the provided content", "options": ["Option 1", "Option 2", "Option 3", "Option 4"], "correct_answer": "Option 1"}, {"question": "Question 2 text based on the provided content", "options": ["Option 1", "Option 2", "Option 3", "Option 4"], "correct_answer": "Option 2"}]}}'

In [82]:
with get_openai_callback() as cb:
    response=generate_evalution_chain(
        {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "RESPONSE_JSON": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

TEXT:<built-in method read of _io.TextIOWrapper object at 0x0000019A97288580>
You are an expert MCQ maker. Given the above text, it is your job to create a quiz of 5 multiple choice questions for AI students in Simple.Make sure the questions are not repeated and check all the questions to conforming.Make sure to format your response like RESPONSE_JSON below and use it as a guide.Ensure to make 5 MCQs
### RESPONSE_JSON
{"quiz": {"text": "{text}", "subject": "{subject}", "tone": "{tone}", "questions": [{"question": "Question 1 text based on the provided content", "options": ["Option 1", "Option 2", "Option 3", "Option 4"], "correct_answer": "Option 1"}, {"question": "Question 2 text based on the provided content", "options": ["Option 1", "Option 2", "Option 3", "Option 4"], "correct_answer": "Option 2"}]}}



RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [81]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:0
Prompt Tokens:0
Completion Tokens:0
Total Cost:0.0


In [83]:
quiz=response.get("quiz")

NameError: name 'response' is not defined

In [84]:
quiz=json.loads(quiz)

NameError: name 'quiz' is not defined

In [85]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

NameError: name 'quiz' is not defined

In [86]:
quiz=pd.DataFrame(quiz_table_data)